# Lab 3.0 Regresión Logística


1. Recordemos que el objetivo de un método de _clasificación_ es asignar una clase o categoría a una observación.
2. Regresión logística es uno de los métodos que podemos usar para esto y es también el método más famoso y usado.
3. *Es* una regresión, pero no dejes que esto te confunda. Estima probabilidades de pertenencia a clases.
4. Este notebook complementa el modulo de Regresión Logística al ilustrar la aplicación en código de lo explicado en clase.

---

<a id='Notebook'></a>
### Estructura del Notebook
- [Importando paquetes](#imporp)
- [Leyendo el dataset](#rds)
    - [Valores faltantes](#msvl)
     
- [Implementación de Regresión Logística](#implementation)
    - [Admisiones](#addmission)
    - [Republicano o Demócrata](#repdemoc)


<a id='imporp'></a>
## Importando paquetes
---

In [ ]:
## paquetes básicos
import numpy as np
import pandas as pd

## paquetes para graficar
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

## paquetes Scikit learn y Statsmodel
from sklearn.linear_model import LogisticRegression, LinearRegression
import statsmodels.api as sm

## Funcionalidades del sistema operativo
import os

!pip install watermark
## Lineas de código necesarias para asegurarse de que los gráficos aparezcan en el notebook, y chequeo de las versiones de los paquetes.
%matplotlib inline
%load_ext watermark
%config InlineBackend.figure_format = 'retina'
%watermark -v -d -a 'Delta Analytics' -p scikit-learn,matplotlib,numpy,pandas

<a id='rds'></a>
## Leyendo el dataset
---
1. En este ejercicio estamos usando el dataset de admisiones.

In [ ]:
data_directory = os.path.join('../data', '')
admission_filepath = os.path.join(data_directory, 'Admissions.csv')
admissions = pd.read_csv(admission_filepath)
admissions.head(3)

In [ ]:
admissions.tail(3)

<a id='msvl'></a>
### Valores faltantes
---
1. De haber, botarlos (no la mejor práctica, pero esta bien por ahora).

In [ ]:
admissions.isnull().sum()

In [ ]:
admissions.dropna(inplace=True)

In [ ]:
admissions.isnull().sum()

In [ ]:
admissions.prestige.value_counts()

<a id='implementation'></a>
## Implementación de Regresión Logística
---


<a id='addmission'></a>
### Admisiones
---

In [ ]:
## Obten estadísticas básicas de tu dataset
admissions.describe()

In [ ]:
## dejemos nuestra columna prestige como enteros
admissions['prestige'] = admissions['prestige'].astype(int)

In [ ]:
admissions.info()

### Si exploras prestige verás que es una variable categórica, la cual puedes convertir a one hot enconding => pandas tiene una función lista para esto
##### https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html

### Unos puntos de precaución:
1. Una vez creado el one hot encoding tenemos que botar la columna original (prestige).
2. Tenemos que botar una de las nuevas columnas
3. Los pasos anteriores son necesarios para evitar dependencias lineales.

In [ ]:
get_dummies = pd.get_dummies(admissions.prestige, prefix="prestige", drop_first=True)
get_dummies.head(4)

In [ ]:
## Agreguemos estas nuevas columnas a nuestro dataset usando concatenación y agreguemos el intercepto
df = pd.concat([admissions, get_dummies], axis=1)
df.drop(['prestige'], inplace=True, axis=1)
df['intercept'] = 1.0

## tenemos ahora un dataset listo para ser analizado
df.head(4)

In [ ]:
'''Define X e y '''
y = df['admit'] 
columns_ = df.columns.tolist()
exclude_col = ['admit']
X = df[[i for i in columns_ if i not in exclude_col]]
print (X.shape, y.shape)

''' divide los datos'''
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=10)

print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

In [ ]:
## construye y ajusta la regresión
logit = sm.Logit(y_train, X_train)
logit_result = logit.fit()
## Imprimamos los resultados
print (logit_result.summary())

In [ ]:
print("Coeficientes")
print(logit_result.params)
print ("\n")
print("Valores p")
print(logit_result.pvalues)
print ("\n")
print("Variables dependientes")
print(logit.endog_names)

## Interpretando los coeficientes de la regresión logística.
### Recuerdas el cociente de probabilidad?
En este caso, el uso del cociente de probabilidad nos ayudará a entender como una unidad de incremento o disminución de cualquiera de las variables afecta la probabilidad de admisión.

In [ ]:
print (np.exp(logit_result.params))

###### Vemos que la probabilidad de admisión podría decrecer en un 46% si el prestigio de la escuela es 2, o en un 23% si el prestigio de la escuela es 3. Estos valores son de nuestro set de entrenamiento, veamos ahora nuestro set de test.

## Prediciendo y Evaluando
Si llamamos el método de predicción, obtendremos las probabilidades predictivas. Pero para hacer una predicción de si el estudiante será admitido o no, debemos convertir estas probabilidades a las etiquetas 1=admitido o 0=no admitido.

In [ ]:
## He aquí las probabilidades predichas
predictions = logit_result.predict(X_test)
print (predictions[:10])

In [ ]:
plt.hist(predictions);

In [ ]:
predictions_nominal = [ 0 if x < 0.5 else 1 for x in predictions]
print (predictions_nominal.count(0))
print (predictions_nominal.count(1))

### Matriz de confusión y reporte de clasificación
---

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
confmat = confusion_matrix(y_true=y_test, y_pred=predictions_nominal)
confusion = pd.DataFrame(confmat, index=['True_Label_0 Rejected', 'True_Label_1 Admitted'],
                         columns=['Predict_Label_0 Rejected', 'Predict_Label_1 Admitted'])

confusion

In [ ]:
print (classification_report(y_test, predictions_nominal, digits=3))

## Implementemos la misma regresión logística usando scikit learn
---

In [ ]:
'''Baseline'''
'''Recuerda 0 es no admitido 1 es admitido'''
print (df['admit'].value_counts(), "\n" )
print ("Si escojemos al azar, %.0f porciento del tiempo escojeremos admitido. " 
        % ((np.mean(df['admit']))*100))

In [ ]:
logistic = LogisticRegression()
logistic.fit(X_train, y_train)

In [ ]:
y_pred=logistic.predict(X_test)
confmat = confusion_matrix(y_true=y_test, y_pred=y_pred)
confusion = pd.DataFrame(confmat, index=['True_Label_0 Rejected', 'True_Label_1 Admitted'],
                         columns=['Predict_Label_0 Rejected', 'Predict_Label_1 Admitted'])

confusion

In [ ]:
print (classification_report(y_test, y_pred, digits=3))

<a id='repdemoc'></a>
### Republicano o Demócrata
---
Para este ejercicio vamos a usar datos de [1984 United States Congressional Voting Records Database] [1]
(echale una mirada al diccionario de los datos) para predecir si un congresista es republicano o demócrata.
[1]: http://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.names "1984 United States Congressional Voting Records Database"

In [ ]:
## Define los nombres de las columnas/variables/características
columns = [
    "class", 
    "handicapped_infants", 
    "water_project_cost", 
    "adoption_of_the_budget_resolution", 
    "physician_fee_freeze",
    "el_salvador_aid",
    "religious_groups_in_schools",
    "anti_satellite_test_ban",
    "aid_to_nicaraguan_contras",
    "mx_missile",
    "immigration",
    "synfuels_corporation_cutback",
    "education_spending",
    "superfund_right_to_sue",
    "crime",
    "duty_free_exports",
    "export_administration_act_south_africa"
]


'''Vamos a leer los datos directamente de la web'''
csv_url = "http://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data"

''' Aquí leemos los datos y creamos una variable binaria 0 para Republicano 1 para Demócrata'''
house_df = pd.read_csv(csv_url, names = columns)

house_df['class'] = house_df['class'].map(lambda value: 0 if value == "republican" else 1 )

In [ ]:
house_df.head(3)

In [ ]:
## Limpiemos el dataset
house_df.replace('?', np.nan, inplace=True)
house_df.ffill(inplace=True)

In [ ]:
## Creemos las variables one hot encoded
df_dummies = pd.get_dummies(house_df)
df_dummies.head(3)

In [ ]:
'''Define X e y '''
y = df_dummies['class'] 
columns_ = df_dummies.columns.tolist()
exclude_col = ['class']

X = df_dummies[[i for i in columns_ if i not in exclude_col]]
print (X.shape, y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=10)
print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

In [ ]:
'''Usa scikit learn'''
r_d_logistic = LogisticRegression()
r_d_logistic.fit(X_train, y_train)

In [ ]:
'''Baseline'''
'''Recuerda  0 es republicana 1 es democrata'''
print (df_dummies['class'].value_counts(), "\n" )
print ("Si escojemos al azar, %.0f porciento del tiempo elijiremos demócrata" 
        % ((np.mean(df_dummies['class']))*100))

In [ ]:
## prediciendo
y_pred=r_d_logistic.predict(X_test)

confmat = confusion_matrix(y_true=y_test, y_pred=y_pred)
confusion = pd.DataFrame(confmat, index=['True_Label_0 Republican', 'True_Label_1 Democrat'],
                         columns=['Predict_Label_0 Republican', 'Predict_Label_1 Democrat'])

confusion

### Obtengamos VP, FP, VN, FN de la matriz de confusión
---

In [ ]:
TP = confusion.loc['True_Label_0 Republican', 'Predict_Label_0 Republican']  

FP = confusion.loc['True_Label_1 Democrat', 'Predict_Label_0 Republican']

TN = confusion.loc['True_Label_1 Democrat', 'Predict_Label_1 Democrat']

FN = confusion.loc['True_Label_0 Republican', 'Predict_Label_1 Democrat']

values = sorted(zip(['True Positives','False Positives','True Negatives','False Negatives'], [TP, FP, TN, FN]))
values

### Calcula exactitud, Razón de error, Precisión,  y Recall
---

In [ ]:
## Exactitud
## Que tan seguido el clasificador está en lo correcto?
from sklearn.metrics import accuracy_score

acc = accuracy_score(y_test, y_pred)
print ("Accuracy score: %.3f" %(acc*100))

In [ ]:
## Razón de error 
## Que tan seguido el clasificador se equivoca?
print ("Error rate: %.3f" % (((FP + FN))/ float(len(y_test))*100))

In [ ]:
## Precisión
## La habilidad del clasificador de evitar etiquetar una clase como miembro de otra
from sklearn.metrics import precision_score

pcs = precision_score(y_test, y_pred)
print ("Precision: %.3f" %(pcs*100))

In [ ]:
## Recall
## Recall la habilidad del clasificador de identificar correctamente la clase actual
from sklearn.metrics import recall_score

rcs = recall_score(y_test, y_pred)
print ("Recall: %.3f" % (rcs*100))

In [ ]:
print (classification_report(y_test, y_pred, digits=3))

### ROC and AUC
---

In [ ]:
from sklearn.metrics import roc_curve, auc

# Obten las probabilidades predichas en el set de test
y_pp = r_d_logistic.predict_proba(X_test)[:,1]

# roc_curve devuelve la razón de falsos positivos y verdaderos positivos a medida que el umbral cambia.
# toma como argumentos y y las probabilidades predichas en la clase positiva.
fpr, tpr, _ = roc_curve(y_test, y_pp)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=[9,9])
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc, linewidth=10, color='g')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray', linewidth=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=16)
plt.ylabel('True Positive Rate', fontsize=16)
plt.title('Receiver operating characteristic curve', fontsize=20)
plt.legend(loc="lower right")
plt.show()